In [3]:
import csv
import numpy as np
import matplotlib.pyplot as plt

testsToAccess: list[str] = ["SM_D1_1", "SM_D1_2", "SM_D1_3"]
filesToAccess: list[str] = [("../../data/archives/" + t + "/data.csv") for t in testsToAccess]
sampleStepSize: int = 1000 # 60
startIndex: int = 0

timesteps:list[int] = []
reward: list[list[float]] = []

isFirstParse: bool = True
for file in filesToAccess:
    with open(file) as f:
        plots = csv.reader(f, delimiter=';')
        next(plots)
        for i, row in enumerate(plots):
            if i < startIndex: continue
            if i % sampleStepSize !=0: continue
            if isFirstParse:
                timesteps.append(int(row[0]))
                reward.append([float(row[len(row)-1])])
            else:
                print(i)
                reward[i].append(float(row[len(row)-1]))

    if file == filesToAccess[0]:
        isFirstParse = False

0
1000


IndexError: list index out of range

In [ ]:
rmax: list[float] = [max(n) for n in reward]
rmin: list[float] = [min(n) for n in reward]
ravg: list[float] = [(sum(n)/len(n)) for n in reward]

In [ ]:
plt.scatter(x=timesteps, y=rmax, color="g")
plt.sca
plt.xlabel("timestep")
plt.ylabel("reward")
plt.title("Dataset")

""" z = np.polyfit(timesteps, reward, 4)
p = np.poly1d(z) """
plt.grid(linestyle="--", linewidth=0.5)
# plt.plot(timesteps, p(timesteps), "r")